In [21]:
%pip install optuna==3.6.1

  Attempting uninstall: optuna
    Found existing installation: optuna 4.0.0
    Uninstalling optuna-4.0.0:
      Successfully uninstalled optuna-4.0.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour


/home/baldasseroniluisesteban/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [19]:
base_path = '/home/baldasseroniluisesteban/buckets/b1/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'exp/'
db_path = dataset_path + 'exp/'

df = pd.read_csv(BASE_PATH + 'competencia_02_sin_nulos.csv.gz')
df.head()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4735593 entries, 0 to 4735592
Columns: 155 entries, numero_de_cliente to clase_ternaria
dtypes: float64(91), int64(63), object(1)
memory usage: 5.5+ GB


In [5]:
df.describe()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
count,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,...,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06,4.735593e+06
mean,7.425024e+08,2.020005e+05,9.536704e-01,4.578307e-03,5.521684e-01,4.676512e+01,1.319457e+02,2.257373e+03,2.559108e+04,1.119460e+03,...,-5.708700e-02,6.349712e+00,5.001662e+03,-2.699271e+04,1.598836e+01,3.145102e+03,1.696529e+04,8.414712e+00,-1.231561e-01,4.460567e+03
std,3.082527e+08,7.715598e+01,2.101979e-01,6.750813e-02,5.064184e-01,1.287174e+01,8.385252e+01,6.976118e+03,5.801515e+04,2.542640e+03,...,7.134903e+00,4.998834e+01,2.204006e+04,4.085428e+04,1.775162e+02,2.120056e+03,3.221313e+04,9.677903e+00,4.492652e-01,1.747707e+04
min,2.492211e+08,2.019010e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-4.473306e+05,-1.885714e+06,-1.758280e+05,...,-1.000000e+00,-3.000000e+00,-1.000000e+00,-7.691583e+06,0.000000e+00,1.000000e+00,-1.849980e+06,0.000000e+00,-1.000000e+00,0.000000e+00
25%,5.063210e+08,2.019100e+05,1.000000e+00,0.000000e+00,0.000000e+00,3.700000e+01,6.200000e+01,0.000000e+00,2.922010e+03,1.417300e+02,...,0.000000e+00,1.000000e+00,0.000000e+00,-3.063592e+04,0.000000e+00,1.420000e+03,2.673390e+03,3.000000e+00,0.000000e+00,3.284400e+02
50%,7.037226e+08,2.020060e+05,1.000000e+00,0.000000e+00,1.000000e+00,4.500000e+01,1.240000e+02,1.136710e+03,1.404195e+04,5.997700e+02,...,0.000000e+00,4.000000e+00,0.000000e+00,-1.648543e+04,0.000000e+00,2.913000e+03,8.020180e+03,6.000000e+00,0.000000e+00,1.384140e+03
75%,9.599964e+08,2.021010e+05,1.000000e+00,0.000000e+00,1.000000e+00,5.600000e+01,1.830000e+02,2.763280e+03,3.043619e+04,1.482710e+03,...,0.000000e+00,7.000000e+00,0.000000e+00,-7.956110e+03,0.000000e+00,4.558000e+03,1.845075e+04,1.000000e+01,0.000000e+00,3.648030e+03
max,1.603867e+09,2.021080e+05,1.000000e+00,1.000000e+00,4.000000e+00,1.080000e+02,3.800000e+02,2.832073e+06,5.625540e+06,1.167083e+06,...,2.853120e+03,6.242000e+03,2.639250e+06,4.573480e+05,4.527039e+04,1.279000e+04,4.323422e+06,2.790000e+02,4.700000e+01,8.329719e+06


In [3]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_inicio = 202002
mes_train_final = 202102
mes_test = 202104

# agregue sus semillas
semillas = [127,181,13]

In [5]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Selección de columnas numéricas para la transformación
data_selected = df[['ctarjeta_visa', 'Visa_cadelantosefectivo', 'cdescubierto_preacordado',
                                  'ctrx_quarter', 'ctarjeta_master', 'cproductos', 'Visa_status',
                                  'cpayroll_trx']]


# Instanciación de PolynomialFeatures con grado 3
poly = PolynomialFeatures(degree=3, include_bias=False)

# Aplicar la transformación a los datos de entrenamiento
data_selected_poly = poly.fit_transform(data_selected)

# Crear DataFrame con nuevas características generadas
data_selected_poly_df = pd.DataFrame(data_selected_poly, columns=poly.get_feature_names_out(data_selected.columns))


In [6]:
df = pd.concat([df, data_selected_poly_df], axis=1)

In [7]:
df.shape

(4735593, 319)

In [9]:
df['clase_peso'] = 1.0

df.loc[df['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df.loc[df['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [12]:
import numpy as np

df['clase_binaria1'] = 0
df['clase_binaria2'] = 0
df['clase_binaria1'] = np.where(df['clase_ternaria'] == 'BAJA+2', 1, 0)
df['clase_binaria2'] = np.where(df['clase_ternaria'] == 'CONTINUA', 0, 1)

In [15]:
train_data = df[(mes_train_inicio <= df['foto_mes']) & (df['foto_mes'] <= mes_train_final)]
test_data = df[df['foto_mes'] == mes_test]

In [16]:
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [18]:
import lightgbm as lgb

train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

/home/baldasseroniluisesteban/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [20]:
def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 1000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)
    max_depth = trial.suggest_int('max_depth', 3, 100)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 300)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': max_depth,
        'bagging_freq': bagging_freq,
        'seed': semillas[0],
        'verbose': -1,
        'early_stopping' :30,
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=50,
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )





    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "exp_302_lgbm"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

NameError: name 'optuna' is not defined